In [ ]:
from model_functions import *
from param_functions import *
import numpy as np
import pandas as pd
from IPython import display as ipd
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

%matplotlib qt

--------------

## Passive Along Cable

In [ ]:
therm_scheme_drive = [[False, True, False, True, True], #inner
                  [False, True, False, True, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_flux = [[False, True, False, False, True], #inner
                  [False, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_output = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [ ]:
c_drive = '219-SS-SS'
c_flux = '219-SS-SS'

p_drive = passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c_drive]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c_drive]['tc'], lambda t: 0, coax_co_params[c_drive]['tc']],
        therm_scheme=therm_scheme_drive,
        stage_temps=fridge_ours['temps'])

p_flux = passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c_flux]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c_flux]['tc'], lambda t: 0, coax_co_params[c_flux]['tc']],
        therm_scheme=therm_scheme_flux,
        stage_temps=fridge_ours['temps'])

p_tot = (p_drive + p_flux)/2

a_ac = active_load_AC(stage_labels=fridge_ours['labels'], 
        signal_p=1e-3*10**(-5/10), 
        signal_f=6, 
        att=[0, 20, 0, 20, 20],
        cable_att=coax_co_params[c_drive]['att_4'],
        lengths=fridge_ours['lengths'])

a_dc = active_load_DC(i_in=20e-3, 
                stage_labels=fridge_ours['labels'], 
                att=[0, 20, 0, 0, 0], 
                cable_rho=coax_co_params[c_flux]['rho'], 
                lengths=fridge_ours['lengths'], 
                diameters=coax_co_params[c_flux]['d'])

In [ ]:
display(p_tot, a_ac, a_dc)

In [ ]:
fig, ax = plt.subplots(figsize=(4.2/1.5, 10.6/1.5))

ax.barh([4*i+2 for i in range(5, 0, -1)], p_tot.values, height=1, color='navy')
ax.barh([4*i+1 for i in range(5, 0, -1)], a_ac.values*0.33, height=1, color='blue')
ax.barh([4*i for i in range(5, 0, -1)], a_dc.values, height=1, color='royalblue')

ax.set_xlim((1e-10, 10))
ax.set_xscale('log')

# ax.set_xticks([10**i for i in range(-9, 1)], minor=True)
ax.set_xticks([10**i for i in range(-9, 1, 3)], minor=False)
ax.set_xticks([10**i for i in range(-9, 1)], minor=True)
ax.tick_params(which='minor', labelsize=0.0)
ax.tick_params(which='minor', labelcolor='white')
ax.set_xlabel("Heat Dumped on Stage\nPer Line (W)")
ax.grid(True, 'major', 'x', linewidth=1, alpha=0.6)
ax.grid(True, 'minor', 'x', linewidth=0.5, alpha=0.6)

ax2 = ax.twiny()
ax2.set_xscale('log')
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks([10**i for i in range(-9, 1, 3)], minor=False)
ax2.set_xticks([10**i for i in range(-9, 1)], minor=True)
ax2.tick_params(which='minor', labelsize=0.0)
ax2.tick_params(which='minor', labelcolor='white')

ax.invert_xaxis()
ax2.invert_xaxis()
ax.set_yticks([])
# ax.legend(['Heat Conduction', 'Signal Absorption (AC)', 'Signal Absorption (DC)'])
fig.set_tight_layout(True)

plt.savefig('Poster Heat1.png', transparent=True)

plt.show()

In [ ]:
ax.set_minor_locator

In [ ]:
c_p = np.array(fridge_ours['cool_p'])
load_matrix_SS = pd.DataFrame(
    [p_drive*14/c_p, p_flux*14/c_p, p_out*4/c_p, a_ac*14*0.33/c_p, a_dc*14*1/c_p], 
    index=['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)']
)
load_matrix_SS

## Noise Alongside Cable

In [ ]:
h = 6.626e-34
k_B = 1.381e-23
Planck = lambda T, f: h*f / ( np.exp( h*f / (k_B*T) ) - 1 )

f = np.logspace(2, 16, 400)
A = [0, 20, 0, 20, 20]

n_in    = [Planck(300, f)]
n_from  = [Planck(t, f) for t in fridge_ours['temps']]

for i in range(5):
    n_in.append(
        n_in[i] * 10**(-A[i]/10) + (1-10**(-A[i]/10)) * n_from[i]
    )

In [ ]:
# Just noise spectrum

# fig, axs = plt.subplots(5, 1, sharex=True, figsize=(4.2, 10.6))
fig, axs = plt.subplots(5, 1, sharex=True, figsize=(4.2/1.4, 10.6/1.4))
axs2 = [ax.twinx() for ax in axs]

for i in range(5):
    axs[i].plot(f, n_in[i], lw=2.5, c='navy')
    axs[i].plot(f, n_from[i], lw=2.5, c='blueviolet')
    axs[i].plot(f, n_in[i+1], lw=2.5, c='royalblue')
        
    axs[i].set_yscale('log')
    axs[i].set_ylim((1e-27, 1e-19))
    # axs[i].set_yticks(np.logspace(-24, -21, 6))
    axs[i].grid(True, 'both', 'x')

    axs[i].axvline(5e9, 0, 1, alpha=0.4, c='mediumslateblue', lw=7)

    axs2[i].set_yscale('log')
    axs2[i].set_ylim((1e-27, 1e-19))
    axs2[i].set_yticks([1e-27, 1e-23, 1e-19], minor=False)
    axs2[i].set_yticks([1e-25, 1e-21], minor=True)
    axs2[i].tick_params(which='minor', labelsize=0.0)
    axs2[i].tick_params(which='minor', labelcolor='white')
    axs2[i].grid(True, 'both', 'y', alpha=0.6)
    axs[i].set_yticks([])

axs[-1].set_xscale('log')
axs[-1].set_xlim((10**(7.5), 10**(14.5)))
axs[-1].set_xticks([10**i for i in [8, 11, 14]], minor=False)
axs[-1].set_xticks([10**i for i in range(8, 15)], minor=True)
axs[-1].tick_params(which='minor', labelsize=0.0)
axs[-1].tick_params(which='minor', labelcolor='white')

ax2 = axs[0].twiny()
ax2.set_xscale('log')
ax2.set_xlim((10**(7.5), 10**(14.5)))
ax2.set_xticks([10**i for i in [8, 11, 14]], minor=False)
ax2.set_xticks([10**i for i in range(8, 15)], minor=True)
ax2.tick_params(which='minor', labelsize=0.0)
ax2.tick_params(which='minor', labelcolor='white')

axs[-1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)

# axs2[2].set_ylabel('Noise Power (W/Hz)')

plt.savefig('Poster Noise1.png', transparent=True)

plt.show()

## Cable Type Outcome

In [ ]:
therm_scheme_drive = [[False, True, False, True, True], #inner
                  [False, True, False, True, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_flux = [[False, True, False, False, True], #inner
                  [False, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_output = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [ ]:
def t_maker(point1, point2):

    def t_stage(power):
        m = (point1[1] - point2[1]) / (point1[0] - point2[0])
        return m * (power - point1[0]) + point1[1]
    
    return t_stage

#point for interpolation of the temperature (excluding the still plate)
points = [
    ((5, 41.9), (10, 46)),
    ((0, 3.16), (0.35, 3.7)),
    ((240e-6, 0.139), (300e-6, 0.148)),
    ((0, 0.007), (20e-6, 0.019))
]

t_stages = [t_maker(p[0], p[1]) for p in points]

In [ ]:
cables = ['119-SS-SS', '119-CuNi-CuNi', '219-SS-SS', '219-CuNi-CuNi']

data = []

for c in cables:

    c_drive = c
    c_flux = c
    c_out_upper = '119-AgCuNi-CuNi'
    c_out_lower = '119-NbTi-NbTi'

    p_drive = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_drive]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_drive]['tc'], lambda t: 0, coax_co_params[c_drive]['tc']],
            therm_scheme=therm_scheme_drive,
            stage_temps=fridge_ours['temps'])

    p_flux = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_flux]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_flux]['tc'], lambda t: 0, coax_co_params[c_flux]['tc']],
            therm_scheme=therm_scheme_flux,
            stage_temps=fridge_ours['temps'])

    p_out_lower = passive_load(stage_labels=fridge_100q['labels'], 
            diameters=coax_co_params[c_out_lower]['d'],
            lengths=fridge_100q['lengths'],
            therm_cond=[coax_co_params[c_out_lower]['tc'], lambda t: 0, coax_co_params[c_out_lower]['tc']],
            therm_scheme=therm_scheme_output,
            stage_temps=fridge_ours['temps'])

    p_out_upper = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_out_upper]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_out_upper]['tc'], lambda t: 0, coax_co_params[c_out_upper]['tc']],
            therm_scheme=therm_scheme_output,
            stage_temps=fridge_ours['temps'])

    p_out = p_out_upper[0:2].append(p_out_lower[2:])

    a_ac = active_load_AC(stage_labels=fridge_ours['labels'], 
            signal_p=1e-3*10**(-5/10), 
            signal_f=6, 
            att=[0, 20, 0, 20, 20],
            cable_att=coax_co_params[c_drive]['att_4'],
            lengths=fridge_ours['lengths'])

    a_dc = active_load_DC(i_in=20e-3, 
                    stage_labels=fridge_ours['labels'], 
                    att=[0, 20, 0, 0, 0], 
                    cable_rho=coax_co_params[c_flux]['rho'], 
                    lengths=fridge_ours['lengths'], 
                    diameters=coax_co_params[c_flux]['d'])
    
    total = p_drive*14 + p_flux*14 + p_out*4 + a_ac*0.33*14 + a_dc*14

    total.iloc[0] = t_stages[0](total.iloc[0])
    total.iloc[1] = t_stages[1](total.iloc[1])
    total.iloc[2] = 0
    total.iloc[3] = t_stages[2](total.iloc[3])
    total.iloc[4] = t_stages[3](total.iloc[4])

    data.append(total)
    

In [ ]:
data_format = pd.DataFrame(data, index=cables).iloc[:, [0, 1, 3, 4]]
data_format

In [ ]:
str(1+1 if 1 < 0 else 2)

In [ ]:
temps = [50, 4, 0.150, 0.020]
t_str = ['50K', '4K', '150mK', '20mK']

fig, axs = plt.subplots(1, 4, figsize=(7.2/1.4, 3.3/1.4))

for i in range(4):
    axs[i].bar([1, 2, 3, 4], data_format.iloc[:, i].values/temps[i], width=1, color=['navy', 'blue', 'royalblue', 'cornflowerblue'])
    
    axs[i].set_xlim(-0.5, 5.5)
    axs[i].set_ylim(0, 2)

    axs[i].set_xticks([])
    axs[i].set_yticks([0])

    axs[i].axhline(1, 0.083, 0.92, ls='--', lw=1, c='r')
    axs[i].annotate(t_str[i], (2.7, 1.02), color='red', fontsize=8)
    
    axs[i].set_xlabel('Stage ' + str(i+1 if i<2 else i+2))

fig.set_tight_layout(True)

# axs2[2].set_ylabel('Noise Power (W/Hz)')

plt.savefig('CableComp.png', transparent=True)

plt.show()

## Attenuation Config Comparison

In [ ]:
def config2therm(config):
    therm = [[not (a < 0.001) for a in config]]*2 + [[True, True, True, True, True]]
    return therm

def t_maker(point1, point2):

    def t_stage(power):
        m = (point1[1] - point2[1]) / (point1[0] - point2[0])
        return m * (power - point1[0]) + point1[1]
    
    return t_stage

#point for interpolation of the temperature (excluding the still plate)
points = [
    ((5, 41.9), (10, 46)),
    ((0, 3.16), (0.35, 3.7)),
    ((240e-6, 0.139), (300e-6, 0.148)),
    ((0, 0.007), (20e-6, 0.019))
]

t_stages = [t_maker(p[0], p[1]) for p in points]

In [ ]:
therm_scheme_flux = [[False, True, False, False, True], #inner
                  [False, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_output = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [ ]:
configs = [
    [0, 0, 0, 0, 60],
    [0, 0, 0, 30, 30],
    [0, 20, 0, 20, 20],
    [10, 10, 0, 20, 20]
]

data = []

for conf in configs:

    therm_scheme_drive = config2therm(conf)

    c_drive = '119-CuNi-CuNi'
    c_flux = '119-CuNi-CuNi'
    c_out_upper = '119-AgCuNi-CuNi'
    c_out_lower = '119-NbTi-NbTi'

    p_drive = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_drive]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_drive]['tc'], lambda t: 0, coax_co_params[c_drive]['tc']],
            therm_scheme=therm_scheme_drive,
            stage_temps=fridge_ours['temps'])

    p_flux = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_flux]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_flux]['tc'], lambda t: 0, coax_co_params[c_flux]['tc']],
            therm_scheme=therm_scheme_flux,
            stage_temps=fridge_ours['temps'])

    p_out_lower = passive_load(stage_labels=fridge_100q['labels'], 
            diameters=coax_co_params[c_out_lower]['d'],
            lengths=fridge_100q['lengths'],
            therm_cond=[coax_co_params[c_out_lower]['tc'], lambda t: 0, coax_co_params[c_out_lower]['tc']],
            therm_scheme=therm_scheme_output,
            stage_temps=fridge_ours['temps'])

    p_out_upper = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_out_upper]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_out_upper]['tc'], lambda t: 0, coax_co_params[c_out_upper]['tc']],
            therm_scheme=therm_scheme_output,
            stage_temps=fridge_ours['temps'])

    p_out = p_out_upper[0:2].append(p_out_lower[2:])

    a_ac = active_load_AC(stage_labels=fridge_ours['labels'], 
            signal_p=1e-3*10**(-5/10), 
            signal_f=6, 
            att=conf,
            cable_att=coax_co_params[c_drive]['att_4'],
            lengths=fridge_ours['lengths'])

    a_dc = active_load_DC(i_in=20e-3, 
                    stage_labels=fridge_ours['labels'], 
                    att=[0, 20, 0, 0, 0], 
                    cable_rho=coax_co_params[c_flux]['rho'], 
                    lengths=fridge_ours['lengths'], 
                    diameters=coax_co_params[c_flux]['d'])
    
    total = p_drive*14 + p_flux*14 + p_out*4 + a_ac*0.33*14 + a_dc*14

    total.iloc[0] = 0
    total.iloc[1] = 0
    total.iloc[2] = 0
    total.iloc[3] = t_stages[2](total.iloc[3])
    total.iloc[4] = t_stages[3](total.iloc[4])

    data.append(total.iloc[-2:])

In [ ]:
data_format = pd.DataFrame(data) 
data_format

In [ ]:
h = 6.626e-34
k_B = 1.381e-23
n_BE = lambda T, f: 1 / ( np.exp( h*f / (k_B*T) ) - 1 )

configs = [
    [0, 0, 0, 0, 60],
    [0, 0, 0, 30, 30],
    [0, 20, 0, 20, 20],
    [10, 10, 0, 20, 20]
]
n_data = []

for conf in configs:
    n_bar = n_BE(300, 6e9)
    for i in range(5):
        n_bar = n_bar * 10**(-conf[i]/10) + (1-10**(-conf[i]/10)) * n_BE(fridge_ours['temps'][i], 6e9)
    n_data.append(n_bar)
n_data

In [ ]:
T_data = 1/(np.array(n_data)+1)
T_data

In [ ]:
fig, ax = plt.subplots(figsize=(4.8/1.5, 3/1.5))

ax.bar([3*i+1 for i in range(4)], data_format.iloc[:, 0].values/0.150, width=1, color='navy')
ax.bar([3*i+2 for i in range(4)], data_format.iloc[:, 1].values/0.02, width=1, color='royalblue')
ax.plot([1.5, 4.5, 7.5, 10.5], T_data*4, c='blueviolet')

ax.set_ylim(0, 5)
ax.set_xticks([1.5, 4.5, 7.5, 10.5])
ax.set_xticklabels(['C1', 'C2', 'C3', 'C4'])

ax2 = ax.twinx()
ax2.set_ylim(0, 5)

ax.set_yticks([0])
ax2.set_yticks([0])

ax.axhline(1, 0.5/12, 11.5/12, ls='--', lw=1, c='r')
ax.annotate('150mK / 20mK', (7.8, 1.04), color='red', fontsize=8)

plt.savefig('ConfigComp.png', transparent=True)

plt.show()

In [ ]:
data_format.iloc[:, 1]/0.02